In [1]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from itertools import combinations 
from numpy import savetxt
import random
from sklearn.model_selection import KFold

2022-10-24 18:01:55.597234: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-24 18:01:55.597256: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import PolynomialFeatures
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout, Input
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras

In [3]:
cols = [
    'carga_subrede_138kv',
    'pot_eolica_subrede_138kv',
    'pot_solar_subrede_138kv',
    'carga_subrede_230kv',
    'pot_eolica_subrede_230kv',
    'pot_solar_subrede_230kv'
]
input_df = pd.read_csv("data/input_data_nn.csv", sep=";")[cols]
output_df = pd.read_csv("data/output_data_nn.csv", sep=";")
# df = pd.concat([input_df, output_df], axis=1)
# df = df.loc[~(df==0).all(axis=1)].reset_index()

In [4]:
def augment(x, y, augmenter = None):
    x, augmenter = _add_interactions(x, augmenter)
    return x, y, augmenter, x.columns

def _add_interactions(df, augmenter):
    combos = list(combinations(list(df.columns), 2))
    colnames = list(df.columns) + ['_'.join(x) for x in combos]

    if augmenter:
        df = augmenter.transform(df)
    else:
        augmenter = PolynomialFeatures(interaction_only=True, include_bias=False)
        df = augmenter.fit_transform(df)

    df = pd.DataFrame(df)
    df.columns = colnames
    
    noint_indicies = [i for i, x in enumerate(list((df == 0).all())) if x]
    df = df.drop(df.columns[noint_indicies], axis=1)
    
    return df, augmenter

In [5]:
def run_main(train_X, test_X, train_Y, test_Y, cur_fold, cur_run):
    data = [train_X, test_X, train_Y, test_Y]
    predicted_df = pd.DataFrame(columns=test_Y.columns, index=test_Y.index.values)
    new_df = test_Y.copy()

    cols_pred = ['tensao_barramento11']
    model_id = 'tensao_11'
    predicted_11 = run(new_df, cols_pred, data, cur_fold, cur_run)

    cols_pred = ['tensao_barramento12']
    model_id = 'tensao_12'
    predicted_12 = run(new_df, cols_pred, data, cur_fold, cur_run)

    cols_pred = ['tensao_barramento24']
    model_id = 'tensao_24'
    predicted_24 = run(new_df, cols_pred, data, cur_fold, cur_run)
    
    new_df['tensao_barramento11'] = predicted_11
    new_df['tensao_barramento12'] = predicted_12
    new_df['tensao_barramento24'] = predicted_24
    predicted_df['tensao_barramento11'] = predicted_11
    predicted_df['tensao_barramento12'] = predicted_12
    predicted_df['tensao_barramento24'] = predicted_24
        
    cols_pred = ['pot_ativa_inj_barramento11']
    model_id = 'pot_ativa_inj_barramento11'
    predictions = run(new_df, cols_pred, data, cur_fold, cur_run)
    predicted_df[model_id] = predictions
    
    cols_pred = ['pot_ativa_inj_barramento12']
    model_id = 'pot_ativa_inj_barramento12'
    predictions = run(new_df, cols_pred, data, cur_fold, cur_run)
    predicted_df[model_id] = predictions
    
    cols_pred = ['pot_ativa_inj_barramento24']
    model_id = 'pot_ativa_inj_barramento24'
    predictions = run(new_df, cols_pred, data, cur_fold, cur_run)
    predicted_df[model_id] = predictions

    cols_pred = ['pot_reativa_inj_barramento11']
    model_id = 'pot_reativa_inj_barramento11'
    predictions = run(new_df, cols_pred, data, cur_fold, cur_run)
    predicted_df[model_id] = predictions
    
    cols_pred = ['pot_reativa_inj_barramento12']
    model_id = 'pot_reativa_inj_barramento12'
    predictions = run(new_df, cols_pred, data, cur_fold, cur_run)
    predicted_df[model_id] = predictions
    
    cols_pred = ['pot_reativa_inj_barramento24']
    model_id = 'pot_reativa_inj_barramento24'
    predictions = run(new_df, cols_pred, data, cur_fold, cur_run)
    predicted_df[model_id] = predictions    

    predicted_df.to_csv(f'results/{cur_run}-{cur_fold}.csv')

In [6]:
def run(df, cols_pred, data, idx, cur_run):
    trainX, testX, trainY, testY = data

    trainY = trainY[cols_pred]
    testY = testY[cols_pred]

    scaler = StandardScaler()
    trainX = scaler.fit_transform(trainX)
    testX = scaler.transform(testX)
    
    print(f" ========== MODEL {cols_pred[0]} {idx} {cur_run} ============")
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=10)

    model = Sequential()
    model.add(Input(shape=(trainX.shape[1],)))
    model.add(Dense(trainX.shape[-1] ** 2, activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(Dense(trainX.shape[-1], activation = 'relu')) 
    model.add(Dropout(0.2))
    model.add(Dense(len(cols_pred)))

    model.compile(loss = 'mae', optimizer = 'adam')
    model.fit(
        trainX, trainY,
        batch_size = 10, epochs = 20,
        callbacks = [es],
        validation_data = (testX, testY),
        verbose=1
    )

    _predictions = model.predict(testX)
    rmse = sqrt(mean_squared_error(testY, _predictions))
    print(f"RMSE: {rmse}")
    
    return _predictions  

In [70]:
for i in range(1, 2):
    cur_fold = 1
    kfold = KFold(n_splits=10, shuffle=True)

    X, Y, _, _, = augment(input_df, output_df)
    for train_ix, test_ix in kfold.split(X, Y):
        train_X, test_X = X.loc[train_ix], X.loc[test_ix]
        train_y, test_y = Y.loc[train_ix], Y.loc[test_ix]

        run_main(train_X, test_X, train_y, test_y, cur_fold, i)

        cur_fold += 1

 ========== MODEL tensao_barramento11 1 1 ============
Epoch 1/20
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
766/789 [============================>.] - ETA: 0s - loss: 0.1750WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f5e86817820> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRA

789/789 [==============================] - 2s 2ms/step - loss: 18.3108 - val_loss: 6.3739
Epoch 2/20
789/789 [==============================] - 1s 1ms/step - loss: 9.5068 - val_loss: 4.2964
Epoch 3/20
789/789 [==============================] - 1s 1ms/step - loss: 8.8961 - val_loss: 5.1578
Epoch 4/20
789/789 [==============================] - 1s 2ms/step - loss: 8.6147 - val_loss: 4.0055
Epoch 5/20
789/789 [==============================] - 1s 2ms/step - loss: 8.6184 - val_loss: 3.0176
Epoch 6/20
789/789 [==============================] - 1s 2ms/step - loss: 8.6054 - val_loss: 3.5196
Epoch 7/20
789/789 [==============================] - 1s 2ms/step - loss: 8.6573 - val_loss: 2.5722
Epoch 8/20
789/789 [==============================] - 1s 2ms/step - loss: 8.5769 - val_loss: 2.7815
Epoch 9/20
789/789 [==============================] - 1s 1ms/step - loss: 8.6190 - val_loss: 4.3250
Epoch 10/20
789/789 [==============================] - 1s 1ms/step - loss: 8.4258 - val_loss: 1.9236
Epoch 11/

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
778/789 [============================>.] - ETA: 0s - loss: 17.6458WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f5e864e33a0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast

Epoch 3/20
789/789 [==============================] - 1s 1ms/step - loss: 4.0836 - val_loss: 2.7448
Epoch 4/20
789/789 [==============================] - 1s 1ms/step - loss: 3.9767 - val_loss: 2.6138
Epoch 5/20
789/789 [==============================] - 1s 1ms/step - loss: 3.9333 - val_loss: 2.6893
Epoch 6/20
789/789 [==============================] - 1s 1ms/step - loss: 3.7765 - val_loss: 2.4200
Epoch 7/20
789/789 [==============================] - 1s 1ms/step - loss: 3.7379 - val_loss: 2.9073
Epoch 8/20
789/789 [==============================] - 1s 1ms/step - loss: 3.6616 - val_loss: 2.0598
Epoch 9/20
789/789 [==============================] - 1s 1ms/step - loss: 3.5490 - val_loss: 2.4202
Epoch 10/20
789/789 [==============================] - 1s 1ms/step - loss: 3.5105 - val_loss: 2.3854
Epoch 11/20
789/789 [==============================] - 1s 1ms/step - loss: 3.4499 - val_loss: 2.1688
Epoch 12/20
789/789 [==============================] - 1s 1ms/step - loss: 3.4277 - val_loss: 2.09

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
768/789 [============================>.] - ETA: 0s - loss: 0.1838WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f5e8728be50> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast'

...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
774/789 [============================>.] - ETA: 0s - loss: 9.7417WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f5e86b78e50> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast'

Epoch 3/20
789/789 [==============================] - 1s 1ms/step - loss: 0.0443 - val_loss: 0.0145
Epoch 4/20
789/789 [==============================] - 1s 1ms/step - loss: 0.0126 - val_loss: 0.0052
Epoch 5/20
789/789 [==============================] - 1s 1ms/step - loss: 0.0083 - val_loss: 0.0050
Epoch 6/20
789/789 [==============================] - 1s 1ms/step - loss: 0.0079 - val_loss: 0.0049
Epoch 7/20
789/789 [==============================] - 1s 1ms/step - loss: 0.0077 - val_loss: 0.0042
Epoch 8/20
789/789 [==============================] - 1s 2ms/step - loss: 0.0076 - val_loss: 0.0061
Epoch 9/20
789/789 [==============================] - 1s 2ms/step - loss: 0.0075 - val_loss: 0.0046
Epoch 10/20
789/789 [==============================] - 1s 2ms/step - loss: 0.0075 - val_loss: 0.0045
Epoch 11/20
789/789 [==============================] - 1s 2ms/step - loss: 0.0072 - val_loss: 0.0043
Epoch 12/20
789/789 [==============================] - 1s 2ms/step - loss: 0.0073 - val_loss: 0.00

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
763/789 [============================>.] - ETA: 0s - loss: 19.7800WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f5e8796f430> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast

789/789 [==============================] - 1s 2ms/step - loss: 8.3597 - val_loss: 6.1759
Epoch 3/20
789/789 [==============================] - 1s 2ms/step - loss: 6.7596 - val_loss: 5.2762
Epoch 4/20
789/789 [==============================] - 1s 2ms/step - loss: 6.1437 - val_loss: 4.0401
Epoch 5/20
789/789 [==============================] - 1s 2ms/step - loss: 5.6833 - val_loss: 4.0836
Epoch 6/20
789/789 [==============================] - 1s 2ms/step - loss: 5.5584 - val_loss: 3.7814
Epoch 7/20
789/789 [==============================] - 1s 2ms/step - loss: 5.4212 - val_loss: 3.8203
Epoch 8/20
789/789 [==============================] - 1s 1ms/step - loss: 5.2076 - val_loss: 3.2242
Epoch 9/20
789/789 [==============================] - 1s 2ms/step - loss: 5.0515 - val_loss: 3.3502
Epoch 10/20
789/789 [==============================] - 1s 2ms/step - loss: 4.9816 - val_loss: 3.3761
Epoch 11/20
789/789 [==============================] - 1s 2ms/step - loss: 4.9064 - val_loss: 3.9715
Epoch 12/

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
781/789 [============================>.] - ETA: 0s - loss: 5.8299WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f5e86955550> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast'

In [29]:
# indexes = np.array(df.index.tolist())
# random.shuffle(indexes)
# ids_to_pred = np.split(indexes, 3)

# results_all = []
# for i in range(1):
#     results_per_ids = []
#     for idx2 in range(len(ids_to_pred)):
#         results = []
#         run_main(df, i, idx2, ids_to_pred[idx2])
#         results_per_ids.append(results)
        
#     results_all.append(results_per_ids)

 ========== MODEL tensao_11 0 ============
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When 

KeyboardInterrupt: 

In [21]:
for _result in results_all:
    print(results_all.index(_result))
    for fold in _result:
        print(f"{_result.index(fold)}: {sum(fold)}")

0
0: 32.33497086128333
1: 36.889005154036376
2: 36.68749314981343
3: 31.932388827029182
1
0: 32.26412368037137
1: 31.990872924834257
2: 36.0298650015969
3: 32.30431549681663
2
0: 31.72844917901247
1: 29.84034210963254
2: 34.37873809844469
3: 38.184718100379435
3
0: 34.15181711482428
1: 30.558933149246954
2: 30.1250874789621
3: 31.1846092629998
4
0: 34.86386086977219
1: 28.93438134306144
2: 36.44611309999486
3: 33.4754096497843
5
0: 37.116041669193386
1: 37.296140359486756
2: 30.919280629845613
3: 39.44656374003831
6
0: 31.38229179727558
1: 30.653259296110438
2: 31.444674673125057
3: 36.76260480496647
7
0: 28.349547201857266
1: 33.73417886152881
2: 39.737148167795624
3: 37.59890026817541
8
0: 29.559890665146096
1: 40.83094120187875
2: 26.866222288396877
3: 37.813916555230335
9
0: 38.21746676037965
1: 37.003628372655385
2: 47.06004230959253
3: 34.13258223262406


In [26]:
print(f"Fold 0: {results_all[7][0]}")
print(f"Fold 1: {results_all[4][1]}")
print(f"Fold 2: {results_all[8][2]}")
print(f"Fold 3: {results_all[3][3]}")

Fold 0: [0.005618806104382909, 0.006374496128408189, 0.005003273984089787, 3.4735610445018055, 5.057355698378199, 3.6107973146928116, 7.466659649968084, 6.293739685672916, 2.4304372324265673]
Fold 1: [0.004912833436191945, 0.006315874222206748, 0.0050400543592956865, 5.882427740243566, 3.5072674248462787, 3.786373119903367, 6.751693446504084, 6.168161926473203, 2.822188923073245]
Fold 2: [0.004500589408310584, 0.004620877592182861, 0.005015842571451259, 3.953451113791048, 4.887246335988537, 4.617532544758241, 5.595597117037946, 5.241330146546544, 2.5569277207026158]
Fold 3: [0.005904257261356303, 0.004717165985727671, 0.005687405161361025, 3.2969980512393264, 5.985023419883928, 3.5217441168282497, 8.310837484354026, 6.692027891640731, 3.3616694706450936]


In [32]:
df_7_0 = pd.read_csv('7-0.csv')
df_7_0 = df_7_0.rename(columns={"Unnamed: 0": "id"})

df_4_1 = pd.read_csv('4-1.csv')
df_4_1 = df_4_1.rename(columns={"Unnamed: 0": "id"})

df_8_2 = pd.read_csv('8-2.csv')
df_8_2 = df_8_2.rename(columns={"Unnamed: 0": "id"})

df_3_3 = pd.read_csv('3-3.csv')
df_3_3 = df_3_3.rename(columns={"Unnamed: 0": "id"})

In [36]:
df_best_predicted = pd.concat([df_7_0, df_4_1, df_8_2, df_3_3])
df_best_predicted = df_best_predicted.sort_values(by=['id'])

In [41]:
df_best_predicted.columns

Index(['id', 'tensao_barramento11', 'tensao_barramento12',
       'tensao_barramento24', 'pot_ativa_inj_barramento11',
       'pot_ativa_inj_barramento12', 'pot_ativa_inj_barramento24',
       'pot_reativa_inj_barramento11', 'pot_reativa_inj_barramento12',
       'pot_reativa_inj_barramento24'],
      dtype='object')